<a href="https://colab.research.google.com/github/ShreyasP20/Machine_Learning/blob/main/Logistic_Regression_For_prediction_of_loan_approval_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import os

df = pd.read_csv('/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv',index_col = 0)
df

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
loan_id,,,,,,,,,,,,
1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...
4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected


In [ ]:
#Data Cleaning, removing of white space
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#Reclassification off graduation, loan status and self employed to 1 and 0
df['graduate_processed'] = df[' education'].map({'Graduate': 1, 'Not Graduate': 0})
df['self_employed_processed'] = df[' self_employed'].map({'Yes': 1, 'No': 0})
df['loan_status_processed'] = df[' loan_status'].map({'Approved': 1, 'Rejected': 0})

#Drop redundant columns not for regression
df.drop(columns=[' education', ' self_employed',' loan_status'],inplace=True)

#Check completeness of data
print('***null data***')
print(df.isnull().sum())
print('***Data post cleaning***')
print(df.shape)
print('***Columns***')
print(df.columns)


***null data***
 no_of_dependents            0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
graduate_processed           0
self_employed_processed      0
loan_status_processed        0
dtype: int64
***Data post cleaning***
(4269, 12)
***Columns***
Index([' no_of_dependents', ' income_annum', ' loan_amount', ' loan_term',
       ' cibil_score', ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', 'graduate_processed',
       'self_employed_processed', 'loan_status_processed'],
      dtype='object')


**Part 1: Logistic Regression**

In [ ]:
#Define X and Y Variables
X = df.drop(columns=['loan_status_processed'])
y = df['loan_status_processed']

#Train Split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.2, random_state = 5)

print('***Train Data***')
print(X_train.shape)
print('***Test Data***')
print(X_test.shape)

***Train Data***
(3415, 11)
***Test Data***
(854, 11)


In [ ]:
#First regression model without manipulation
model1 = LogisticRegression(max_iter=1000,random_state=0).fit(X_train, y_train)
print('Intercept:', model1.intercept_)
print('Coefficients:', model1.coef_)
print('Accuracy:',model1.score(X_test,y_test))

#Accuracy of 71% is not bad!

Intercept: [1.7278216e-06]
Coefficients: [[ 3.63254419e-06 -5.06970945e-07  5.70882453e-08  4.81714292e-06
   4.06628939e-03 -2.83601693e-09  1.73054946e-08  8.05267492e-09
   1.72308935e-08  8.18874335e-07  8.98289454e-07]]
Accuracy: 0.7177985948477752


In [ ]:
#Print confusion matrix
preds = model1.predict(X_test)
confmtrx = np.array(confusion_matrix(y_test, preds))
confusion = pd.DataFrame(confmtrx, index=['approved', 'not_approved'],
columns=['predicted_approved', 'predicted_not_approved'])
confusion

#However, there are around 1/4-1/3 of false positive and false negative

,predicted_approved,predicted_not_approved
approved,124,200
not_approved,41,489


In [ ]:
#odds ratio
oddsratio = pd.DataFrame({ 'Variable':X.columns.tolist(),'Odds_ratio': (np.exp(model1.coef_).T).round(2).flatten().tolist()})
oddsratio

#Dont make sense as all is 1.0. Likely due to not scaling properly

,Variable,Odds_ratio
0,no_of_dependents,1.0
1,income_annum,1.0
2,loan_amount,1.0
3,loan_term,1.0
4,cibil_score,1.0
5,residential_assets_value,1.0
6,commercial_assets_value,1.0
7,luxury_assets_value,1.0
8,bank_asset_value,1.0
9,graduate_processed,1.0


In [ ]:
logit_model=sm.Logit(y_train,sm.add_constant(X_train))
logit_model
result=logit_model.fit()
stats1=result.summary()
print(stats1)

#We can see factor of importance (P>|Z|) are annual income, loan amount, loan term and cibil score!

Optimization terminated successfully.
         Current function value: 0.220519
         Iterations 8
                             Logit Regression Results                            
Dep. Variable:     loan_status_processed   No. Observations:                 3415
Model:                             Logit   Df Residuals:                     3403
Method:                              MLE   Df Model:                           11
Date:                   Mon, 20 Nov 2023   Pseudo R-squ.:                  0.6673
Time:                           14:31:23   Log-Likelihood:                -753.07
converged:                          True   LL-Null:                       -2263.5
Covariance Type:               nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                       -11.3345      0.492    -23

**Part 2 Logistic Regression with Standard scaling**

In [ ]:
#Standard Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test) #not fit to preserve variance

In [ ]:
#Log regression
model2 = LogisticRegression(max_iter=1000,random_state=0).fit(X_train_scaled, y_train)
preds2 = model2.predict(X_test_scaled)

print('Intercept:', model2.intercept_)
print('Coefficients:', model2.coef_)
print('Accuracy:',model2.score(X_test_scaled,y_test))

#much higher score at 92%!

Intercept: [1.75883065]
Coefficients: [[-0.06199442 -1.67704297  1.24200566 -0.84453041  4.15168277  0.02616445
   0.09278711  0.25271997  0.16627169  0.05381782  0.08171827]]
Accuracy: 0.9262295081967213


In [ ]:
#Odd ratio
oddsratio = pd.DataFrame({ 'Variable':X.columns.tolist(),'Odds_ratio': (np.exp(model2.coef_).T).round(2).flatten().tolist()})
oddsratio
# cibil Score has outsized impact to loan approval, follow by annual income and loan amount

,Variable,Odds_ratio
0,no_of_dependents,0.94
1,income_annum,0.19
2,loan_amount,3.46
3,loan_term,0.43
4,cibil_score,63.54
5,residential_assets_value,1.03
6,commercial_assets_value,1.10
7,luxury_assets_value,1.29
8,bank_asset_value,1.18
9,graduate_processed,1.06


In [ ]:
confmtrx = np.array(confusion_matrix(y_test, preds2))
confusion = pd.DataFrame(confmtrx, index=['approved', 'not_approved'],
columns=['predicted_approved', 'predicted_not_approved'])
confusion
#Much better confusion matrix. recall of 91%, Precision 89%

,predicted_approved,predicted_not_approved
approved,289,35
not_approved,28,502


In [ ]:
logit_model=sm.Logit(y_train,sm.add_constant(X_train_scaled))
logit_model
result=logit_model.fit()
stats1=result.summary()
print(stats1)
#Similarly, We can see factor of importance (P>|Z|) are annual income, loan amount, loan term and cibil score!

Optimization terminated successfully.
         Current function value: 0.220519
         Iterations 8
                             Logit Regression Results                            
Dep. Variable:     loan_status_processed   No. Observations:                 3415
Model:                             Logit   Df Residuals:                     3403
Method:                              MLE   Df Model:                           11
Date:                   Mon, 20 Nov 2023   Pseudo R-squ.:                  0.6673
Time:                           14:31:24   Log-Likelihood:                -753.07
converged:                          True   LL-Null:                       -2263.5
Covariance Type:               nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8131      0.094     19.207      0.000       1.628       1.998
x1   

**Part 3 Feature Selection**

In [ ]:
#Logically, worth no matter residential, commercial, luxury or bank asset contribute to personal wealth
df['Total_worth']=df[' residential_assets_value']+df[' commercial_assets_value']+df[' luxury_assets_value']+df[' bank_asset_value']

#drop un-needed columns especially columns that didnt show significance earlier
X=df.drop(columns=[' no_of_dependents', ' residential_assets_value', ' commercial_assets_value',
       'loan_status_processed',' luxury_assets_value',' bank_asset_value','graduate_processed', 'self_employed_processed'])

#split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.2, random_state = 5)

#remaining columns
print(X.columns)

Index([' income_annum', ' loan_amount', ' loan_term', ' cibil_score',
       'Total_worth'],
      dtype='object')


In [ ]:
#Log regression fitting
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
model3 = LogisticRegression(max_iter=1000,random_state=0).fit(X_train_scaled, y_train)
preds3 = model3.predict(X_test_scaled)
#params = model1.get_params()

print('Intercept: ', model3.intercept_)
print('Coefficients: ', model3.coef_)
print('Accuracy: ', model3.score(X_test_scaled,y_test))

#Compared to 92.62%, only minor improvement

Intercept:  [1.75464033]
Coefficients:  [[-1.50005599  1.22766259 -0.83441298  4.13811101  0.30913629]]
Accuracy:  0.927400468384075


In [ ]:
confmtrx = np.array(confusion_matrix(y_test, preds3))
confusion = pd.DataFrame(confmtrx, index=['approved', 'not_approved'],
columns=['predicted_approved', 'predicted_not_approved'])
confusion
#confusion matrix show minor improvement as well

,predicted_approved,predicted_not_approved
approved,289,35
not_approved,27,503


In [ ]:
logit_model=sm.Logit(y_train,sm.add_constant(X_train_scaled))
logit_model
result=logit_model.fit()
stats1=result.summary()
print(stats1)
#Total wealth show borderline significant

Optimization terminated successfully.
         Current function value: 0.221476
         Iterations 8
                             Logit Regression Results                            
Dep. Variable:     loan_status_processed   No. Observations:                 3415
Model:                             Logit   Df Residuals:                     3409
Method:                              MLE   Df Model:                            5
Date:                   Mon, 20 Nov 2023   Pseudo R-squ.:                  0.6658
Time:                           14:31:24   Log-Likelihood:                -756.34
converged:                          True   LL-Null:                       -2263.5
Covariance Type:               nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8070      0.094     19.236      0.000       1.623       1.991
x1   

In [ ]:
oddsratio = pd.DataFrame({ 'Variable':X.columns.tolist(),'Odds_ratio': (np.exp(model3.coef_).T).round(2).flatten().tolist()})
oddsratio
#Cibil has outsized impact

,Variable,Odds_ratio
0,income_annum,0.22
1,loan_amount,3.41
2,loan_term,0.43
3,cibil_score,62.68
4,Total_worth,1.36


**Multicollinearlity test**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# For each X, calculate VIF and save in dataframe
x_temp = sm.add_constant(X_train)

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x_temp.values, i) for i in range(x_temp.values.shape[1])]
vif["features"] = x_temp.columns
print(vif.round(1))

# Income, loan amount and total worth are closely related to each others!
# Side note, When use only loan amount, loan term and cibil score, the prediction went to 93%. But which banker will approve loan without income statement?

   VIF Factor       features
0        20.1          const
1        13.8   income_annum
2         7.2    loan_amount
3         1.0      loan_term
4         1.0    cibil_score
5         7.7    Total_worth
